# Delete Cycle

This notebook permanently deletes active cycle from your Risk Modeling workflow. 

1. Deletes the _Active directory 
2. Removes the cycle, stages and steps from the database

**Note:** This action cannot be reversed!

In [1]:
%load_ext autoreload
%autoreload 2

import shutil
from pathlib import Path
from helpers import cycle, ux, constants
from helpers.context import WorkContext

## Delete Active Cycle

This section will:
1. Display current active cycle information
2. Confirm deletion with double-confirmation prompts
3. Delete the Active_ directory from filesystem
4. Delete the cycle and all related records from database (CASCADE)

In [2]:
ux.header("Current Active Cycle")

# Get active cycle
active_cycle = cycle.get_active_cycle()

if not active_cycle:
    ux.error("No active cycle found.")
    raise Exception("No active cycle to delete")

cycle_name = active_cycle['cycle_name']
cycle_id = active_cycle['id']

# Show current cycle info
ux.success(f"Active cycle: {cycle_name}")
ux.info(f"Cycle ID: {cycle_id}")
ux.info(f"Created: {active_cycle['created_ts']}")

# Show cycle progress
progress_df = cycle.get_cycle_progress(cycle_name)
if not progress_df.empty:
    ux.subheader(f"Current Progress ({len(progress_df)} steps)")
    ux.step_progress(progress_df)
else:
    ux.info("No step execution history")

# Confirm deletion
print()
ux.header("⚠️ CONFIRMATION REQUIRED")
ux.warning(f"You are about to PERMANENTLY DELETE cycle: {cycle_name}")
print()
print("This will:")
print(f"  1. PERMANENTLY DELETE the Active_{cycle_name} directory")
print(f"  2. PERMANENTLY DELETE all database records (stages, steps, runs, batches, jobs, configurations)")
print()
ux.error("THIS ACTION CANNOT BE UNDONE! NO ARCHIVE WILL BE CREATED!")
print()

if not ux.yes_no("Are you sure you want to proceed?"):
    ux.info("Operation cancelled by user")
    raise Exception("User cancelled operation")

# Double confirmation
if not ux.yes_no(f"Final confirmation: PERMANENTLY DELETE '{cycle_name}'?"):
    ux.info("Operation cancelled by user")
    raise Exception("User cancelled operation")

ux.success("Confirmation received. Proceeding with deletion...")

# Step 1: Delete directory
ux.subheader("Step 1: Deleting Cycle Directory")
try:
    active_dir = constants.WORKFLOWS_PATH / f"Active_{cycle_name}"
    
    if active_dir.exists():
        shutil.rmtree(active_dir)
        ux.success(f"✓ Deleted directory: {active_dir.name}")
    else:
        ux.warning(f"Active directory not found: {active_dir.name}")
    
except Exception as e:
    ux.error(f"Failed to delete cycle directory: {str(e)}")
    raise

# Step 2: Delete from database
ux.subheader("Step 2: Deleting Database Records")
try:
    # This will CASCADE delete all related records:
    # - irp_configuration (configurations)
    # - irp_stage (stages)
    # - irp_step (steps)
    # - irp_step_run (step execution history)
    # - irp_batch (batches)
    # - irp_job (jobs)
    success = cycle.delete_cycle(cycle_id)
    
    if success:
        ux.success("✓ Cycle deleted from database (CASCADE deleted all related records)")
    else:
        raise Exception("Failed to delete cycle from database")
    
except Exception as e:
    ux.error(f"Failed to delete cycle: {str(e)}")
    raise

# Completion message
print()
ux.header("✓ DELETION COMPLETE")
ux.success(f"Cycle '{cycle_name}' has been permanently deleted")
print()
print("Next steps:")
print("  - Use 'New Cycle' notebook to create a new cycle")
print("  - Or use 'Replace Active Cycle' if you need the same cycle name")


This will:
  1. PERMANENTLY DELETE the Active_USEQ-202512 directory
  2. PERMANENTLY DELETE all database records (stages, steps, runs, batches, jobs, configurations)



Are you sure you want to proceed? (y/n):  y
Final confirmation: PERMANENTLY DELETE 'USEQ-202512'? (y/n):  y



Next steps:
  - Use 'New Cycle' notebook to create a new cycle
  - Or use 'Replace Active Cycle' if you need the same cycle name


---